In [ ]:
# Import libraries
from financetoolkit import Toolkit

In [ ]:
# Load tickers
tick_path = "./stock_lists/wilshire_5000.txt" 
tickers = []

with open(tick_path, "r") as file:
    
    # Read each line in the file and append it to the list
    for line in file:
        tickers.append(line.strip())

# Print our list of tickers
print(f"Loading data from {len(tickers)} stocks...")

In [ ]:
# Use a private, Basic-Plan api key to fetch all quarterly data between 2001 and 2023.
key = "[API_KEY]"
companies = Toolkit(tickers, api_key=key, start_date='2001-01-01', end_date='2023-01-01', quarterly=True)

In [ ]:
# Get all stats from income statements, balance sheets and cash flows
bal_data = companies.get_balance_sheet_statement(growth=True).stack().unstack(level=1)
cas_data = companies.get_cash_flow_statement(growth=True).stack().unstack(level=1)
inc_data = companies.get_income_statement(growth=True).stack().unstack(level=1)

In [ ]:
# Get all efficiency, liquidity, productivity, solvency and valuation  ratios
eff_ratio = companies.ratios.collect_efficiency_ratios().stack().unstack(level=1)
liq_ratio = companies.ratios.collect_liquidity_ratios().stack().unstack(level=1)
pro_ratio = companies.ratios.collect_profitability_ratios().stack().unstack(level=1)
val_ratio = companies.ratios.collect_valuation_ratios().stack().unstack(level=1)
sol_ratio = companies.ratios.collect_solvency_ratios().stack().unstack(level=1)

# Valuation ratios come out strange, so we fix them manually
val_ratio.index = val_ratio.index.set_names('date', level=1)

In [ ]:
# Create a set of dataframes
dfs = [ bal_data, cas_data, eff_ratio, liq_ratio, pro_ratio ]
master_df = inc_data

In [ ]:
# Concatenate the data frames
for df in dfs:
    master_df = master_df.merge(df, how='outer', left_index=True, right_index=True)

In [ ]:
# Print the head of our final dataset and its shape
print(master_df.head())

In [ ]:
# Output the dataset
inc_data.to_csv('./output/fundamentals_growth.csv')